# Workflow Example with ERA5-Land input convertor
Here we illustrate a complete workflow example using the ERA5 convertor

## Imports

In [1]:
from pathlib import Path

import xarray as xr
import pandas as pd

import valenspy as vp #The Valenspy package
from valenspy.inputconverter_functions import _non_convertor, ERA5Land_to_CF

from yaml import safe_load
from typing import Union, List

In [2]:
#Import Converter - This input converter will not do anything to the data.
ic = vp.InputConverter(ERA5Land_to_CF)


## 1. Convert the data

### Get the filenames of ERA5 data corresponding to CORDEX variable name


In [3]:
machine = "hortense"

# User defined variable, here we look at 2m temperature. 
variable = "hfls"
dataset = "ERA5-Land"
region = "belgium" # "belgium"
time_freq = "daily" ## Important to account here are the postprocessed files at daily time resolution and their naming (eg max daily temperature does not correspond to ERA5 variable name as defined!!)

### Generate the path and filename of obs file

In [4]:
# Get the current file directory and load the CORDEX variables.yml file

# files = Path(__file__).resolve().parent -- this is not working in notebook
src_path = Path("../src/valenspy") ## -- to be removed. 

with open(src_path / "ancilliary_data" / "CORDEX_variables.yml") as file:
    CORDEX_VARIABLES = safe_load(file)

# ERA5-Land uses same lookup table as ERA5
with open(src_path / "ancilliary_data" / Path("ERA5_lookup.yml")) as file:
    obs_LOOKUP = safe_load(file)

In [5]:
# alternative to PATH generator. 

# get path of observational datasets from path settings .yml
with open(src_path / "ancilliary_data" / Path("dataset_PATHS.yml")) as file:
    dataset_PATHS = safe_load(file)

directory = dataset_PATHS[machine][dataset]

# get ERA5 variable corresponding to the requested variable using its look-up table
obs_var = obs_LOOKUP[variable]['obs_name']

# get ERA5 subdirectory
subdirectory = Path(directory + '/' + region +'/'+time_freq+'/'+obs_LOOKUP[variable]['obs_long_name']+'/')

# open the EOBS file for the corresponding variable
files = list(subdirectory.glob("*-"+time_freq+"-*"+obs_LOOKUP[variable]['obs_long_name']+"*20*.nc")) #Select all the netCDF files in the directory
files

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2007.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2008.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2018.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2012.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2021.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200

In [6]:
# test of ERA5 convertor function
ERA5Land_ds = ic.convert_input(files) #Convert the input to the correct format
ERA5Land_ds

The file is NOT ValEnsPy CF compliant.
0.00% of the variables are ValEnsPy CF compliant
NOT ValEnsPy CF compliant: ['hfls']
Unknown to ValEnsPy: ['time_bnds']
The following attributes are missing or incorrect for the variable hfls:
Attribute       Actual                    Expected                 
-----------------------------------------------------------------
units           kg m-2 s-1                W m-2                    


<xarray.Dataset> Size: 66MB
Dimensions:    (time: 8036, bnds: 2, lat: 25, lon: 41)
Coordinates:
  * time       (time) datetime64[ns] 64kB 2000-01-01T11:00:00 ... 2021-12-31T...
  * lon        (lon) float32 164B 2.0 2.125 2.25 2.375 ... 6.625 6.75 6.875 7.0
  * lat        (lat) float32 100B 52.0 51.88 51.75 51.62 ... 49.25 49.12 49.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 129kB dask.array<chunksize=(366, 2), meta=np.ndarray>
    hfls       (time, lat, lon) float64 66MB dask.array<chunksize=(366, 25, 41), meta=np.ndarray>
Attributes:
    CDI:                 Climate Data Interface version 1.9.9 (https://mpimet...
    Conventions:         CF-1.6
    history:             Mon Sep 05 05:25:37 2022: cdo -z zip daymean /scratc...
    frequency:           day
    CDO:                 Climate Data Operators version 1.9.9 (https://mpimet...
    freq:                daily
    region:              belgium
    dataset:             ERA5-Land
    spatial_resolution:  0.1deg

In [6]:
#Multiple variables test.

variables = ["hfls", "tas"]
files=[]
for variable in variables:
    # get ERA5 subdirectory
    subdirectory = Path(directory + '/' + region +'/'+time_freq+'/'+obs_LOOKUP[variable]['obs_long_name']+'/')
    files+=list(subdirectory.glob("*-"+time_freq+"-*"+obs_LOOKUP[variable]['obs_long_name']+"*20*.nc")) #Select all the netCDF files in the directory
files

[PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2007.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2008.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2018.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2012.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200/project_input/External/observations/era5-land/belgium/daily/surface_latent_heat_flux/era5-land-daily-belgium-surface_latent_heat_flux-2021.nc'),
 PosixPath('/dodrio/scratch/projects/2022_200

In [7]:
ERA5Land_ds = ic.convert_input(files) #Convert the input to the correct format
ERA5Land_ds

ValueError: Cannot specify both coords='different' and compat='override'.